In [70]:
import json
import csv
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [71]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "中信兄弟": 1.15,
    "統一7-ELEVEn獅": 1.11,
    "樂天桃猿": 0.96, 
    "富邦悍將": 0.93,
    "味全龍": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"
scheduleRoot = "../schedule/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [72]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

In [73]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [74]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [75]:
team = "富邦悍將"
scheduleList = {}
with open(scheduleRoot + team + "比賽.csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 9):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

# scheduleByWeek

In [76]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Calculate Parameters

In [77]:
# 比賽當天 27 人名單去除投手
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[aTeam]].keys():
        if date > key:
            break
        if key - date < minDelta:
            minDelta = key - date
            playerDay = key
    
    withoutPitcherList = []
    for data in allData[teamNameMap[team]]:
        withoutPitcherList.append(data.name)
    return list(filter(lambda d: d in withoutPitcherList, playerList[teamNameMap[team]][playerDay]))
        

In [78]:
# A_{ij}
def calcA(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        return 1
    else:
        return 0

In [79]:
# F_{ij}
def calcF(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        F = float(player.data["fielding"]["pos"][str(pos)]["FPCT"]) / FPCT_AVG[str(pos)]
        return F
    else:
        return 0

In [80]:
# B_i
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

def calcB(game, player, aMonth):
    if game["date"].month == 4:
        aMonth = "Apr"
    if game["date"].month == 5:
        aMonth = "May"
    if game["date"].month == 6:
        aMonth = "Jun"
    aOppo = game["oppo"]
    aPitcher = game["pitcher"]
    aField = game["field"]

    OPSseason = float(player.data["batting"]["season"]["OPS+"])
    PAseason = int(player.data["batting"]["season"]["PA"])
    
    if "month" in player.data["batting"].keys():
        if aMonth in player.data["batting"]["month"].keys():
            if int(player.data["batting"]["month"][aMonth]["AB"]) != 0:
                OPSmonth = calcOPS(
                    float(player.data["batting"]["month"][aMonth]["OBP"]), 
                    float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
                )
                PAmonth = int(player.data["batting"]["month"][aMonth]["PA"])
            else:
                OPSmonth = OPSseason
                PAmonth = 0
        else:
            OPSmonth = OPSseason
            PAmonth = 0
    else:
        OPSmonth = OPSseason
        PAmonth = 0

    if "field" in player.data["batting"]:
        if aField in player.data["batting"]["field"].keys():
            if int(player.data["batting"]["field"][aField]["AB"]) != 0:
                OPSfield = calcOPS(
                    float(player.data["batting"]["field"][aField]["OBP"]), 
                    float(player.data["batting"]["field"][aField]["TB"]) / float(player.data["batting"]["field"][aField]["AB"])
                )
                PAfield = int(player.data["batting"]["field"][aField]["PA"])
            else:
                OPSfield = OPSseason
                PAfield = 0
        else:
            OPSfield = OPSseason
            PAfield = 0
    else:
        OPSfield = OPSseason
        PAfield = 0
    
    if "vsP" in player.data["batting"].keys():
        if aOppo in player.data["batting"]["vsP"]["data"].keys():
            if aPitcher in player.data["batting"]["vsP"]["data"][aOppo].keys():
                if int(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"]) != 0:
                    OPSvsp = calcOPS(
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["OBP"]), 
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["TB"]) / float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"])
                    )
                    PAvsp = int(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["PA"])
                else:
                    OPSvsp = OPSseason
                    PAvsp = 0
            else:
                OPSvsp = OPSseason
                PAvsp = 0
        else:
            OPSvsp = OPSseason
            PAvsp = 0
    else:
        OPSvsp = OPSseason
        PAvsp = 0

    B = (OPSseason * PAseason + OPSmonth * PAmonth + OPSfield * PAfield + OPSvsp * PAvsp) / (PAseason + PAmonth + PAfield + PAvsp)
    return B

# V_{ij}
def calcV(game, month, player, pos):
    A = calcA(player, pos)
    B = calcB(game, player, month)
    F = calcF(player, pos)
    return (POSITION_WEIGHT[str(pos)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * F) * A

### Integer Program

#### Stage 1

In [97]:
def stage1(aTeam, aGame, aMonth):
    model = gp.Model("model")

    # decision variables
    x = []
    for g in range(len(aGame)):
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        singleGame = {}
        for i in range(len(todayPlayerList)):
            singlePlayer = []
            for j in range(2, 10):
                singlePlayer.append(model.addVar(lb=0, vtype="B", name="x" + str(g) + "-" + todayPlayerList[i] + "-" + str(j)))
            singleGame[todayPlayerList[i]] = singlePlayer
        x.append(singleGame)

    # objective function
    model.setObjective(gp.quicksum(
        gp.quicksum(
            gp.quicksum(
                OPPONENT_WEIGHT[aGame[g]["oppo"]] * calcV(aGame[g], aMonth, list(filter(lambda d: d.name == thisPlayer, allData[teamNameMap[aTeam]]))[0], j) * x[g][thisPlayer][j - 2]
                for j in range(2, 10)
            ) for thisPlayer in findPlayerList(aTeam, aGame[g]["date"])
        ) for g in range(len(aGame))
    ), gp.GRB.MAXIMIZE)

    # constraints
    zSum = 0
    for g in range(len(aGame)):
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        zSum += OPPONENT_WEIGHT[aGame[g]["oppo"]]
        for thisPlayerName in todayPlayerList:
        
            thisPlayer = list(filter(lambda d: d.name == thisPlayerName, allData[teamNameMap[aTeam]]))[0]
                    
            # constraint 2
            model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for j in range(2, 10)) <= 1)

            for j in range(2, 10):
                # constraint 3
                model.addConstr(x[g][thisPlayerName][j - 2] <= calcA(thisPlayer, j))

        for j in range(2, 10):
            # constraint 1
            model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for thisPlayerName in todayPlayerList) <= 1)

    # constraint 4
    for thisPlayer in findPlayerList(aTeam, aGame[0]["date"]):
        if len(aGame) >= 4:
            for n in range(4, len(aGame) + 1):
                model.addConstr(
                    gp.quicksum(
                        x[g][thisPlayer][2 - 2] if thisPlayer in findPlayerList(aTeam, aGame[g]["date"]) else 0
                        for g in range(n - 4, n)
                    ) <= 3
                )

    model.optimize()

    playerName = []
    onPlayer = {}
    for g in range(len(aGame)):
        singleGame = []
        onPlayerTemp = {}
        for j in range(8):
            for name in x[g].keys():
                if x[g][name][j].x == 1:
                    singleGame.append(name)
                    onPlayerTemp[name] = { "field": str(j + 2) }
        playerName.append(singleGame)
        onPlayer[aGame[g]["date"]] = onPlayerTemp
        print("===== complete game", g, "- stage 1 =====")
    
    return playerName, onPlayer, model.objVal, model.objVal / zSum

#### Stage 1.5 - DH

In [82]:
def fillField(aTeam, game, playerName, positionType):
    candidate = list(filter(lambda d: d not in playerName, findPlayerList(aTeam, game["date"])))
    findPlayerList(aTeam, game["date"])
    # 內野手
    if positionType == 1:
        for can in candidate:
            thisPlayerObject = list(filter(lambda d: d.name == can, allData[teamNameMap[aTeam]]))[0]
            for pos in ["3", "4", "5", "6"]:
                if pos in thisPlayerObject.data["fielding"]["pos"].keys():
                    return can
    # 外野手
    else:
        for can in candidate:
            thisPlayerObject = list(filter(lambda d: d.name == can, allData[teamNameMap[aTeam]]))[0]
            for pos in ["7", "8", "9"]:
                if pos in thisPlayerObject.data["fielding"]["pos"].keys():
                    return can

In [83]:
def stageDH(aTeam, aGame, aMonth, playerName, onPlayer):
    for g in range(len(aGame)):
        if len(playerName[g]) < 8:
            onPositionRecord = [False, False, False, False, False, False, False, False]
            for player in onPlayer[aGame[g]["date"]].keys():
                onPositionRecord[int(onPlayer[aGame[g]["date"]][player]["field"]) - 2] = True
            for i in range(len(onPositionRecord)):
                if(onPositionRecord[i] == False):
                    if i <= 4:
                        fill = fillField(aTeam, aGame[g], playerName, 1)
                    else:
                        fill = fillField(aTeam, aGame[g], playerName, 2)
                    onPlayer[aGame[g]["date"]][fill] = { "field": str(i + 2) }
                    playerName[g].append(fill)

        maxB = -1
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        for todayPlayer in todayPlayerList:
            if todayPlayer not in playerName[g]:
                thisPlayerObject = list(filter(lambda d: d.name == todayPlayer, allData[teamNameMap[aTeam]]))[0]
                thisB = calcB(aGame[g], thisPlayerObject, aMonth)
                if thisB > maxB:
                    DHPlayerName = thisPlayerObject.name
                    maxB = thisB
        playerName[g].append(DHPlayerName)
        onPlayer[aGame[g]["date"]][DHPlayerName] = { "field": "DH" }

        print("===== complete game", g, "- stage DH =====")
    return onPlayer

#### Stage 2

In [84]:
def stage2(aTeam, aGame, onPlayer):
    stage2Obj = []
    for g in range(len(aGame)):
        model2 = gp.Model("model2")
        
        allO = []
        y = []
        for j in range(1, 10):
            thisPlayer = list(filter(lambda d: d.name == playerName[g][j - 1], allData[teamNameMap[aTeam]]))[0]
            temp = []
            tempY = []
            for k in range(1, 10):
                if "baorder" in thisPlayer.data["batting"].keys():
                    if str(k) in thisPlayer.data["batting"]["baorder"].keys():
                        thisOrderData = thisPlayer.data["batting"]["baorder"][str(k)]
                        PA = int(thisOrderData["PA"])
                        if PA >= 30:
                            O = (int(thisOrderData["IBB"][1]) * 0.72 + int(thisOrderData["HBP"]) * 0.75 
                            + (int(thisOrderData["H"]) - int(thisOrderData["2B"]) - int(thisOrderData["3B"]) - int(thisOrderData["HR"])) * 0.9
                            + int(thisOrderData["2B"]) * 1.24 + int(thisOrderData["3B"]) * 1.56 + int(thisOrderData["HR"]) * 1.95) / PA
                        else:
                            O = thisPlayer.data["batting"]["season"]["wOBA"]
                    else:
                        O = thisPlayer.data["batting"]["season"]["wOBA"]
                else:
                    O = thisPlayer.data["batting"]["season"]["wOBA"]
                temp.append(O)
                tempY.append(model2.addVar(lb=0, vtype="B", name="y" + str(j) + "-" + str(k)))

            allO.append(temp)
            y.append(tempY)
        
        model2.setObjective(
            gp.quicksum(
                gp.quicksum(
                    allO[j - 1][k - 1] * y[j - 1][k - 1] 
                    for k in range(1, 10)
                ) 
                for j in range(1, 10)
            ),
            gp.GRB.MAXIMIZE
        )

        for k in range(1, 10):
            model2.addConstr(gp.quicksum(y[j - 1][k - 1] for j in range(1, 10)) == 1)
        for j in range(2, 10):
            model2.addConstr(gp.quicksum(y[j - 1][k - 1] for k in range(1, 10)) == 1)

        model2.optimize()
        
        for j in range(1, 10):
            for k in range(1, 10):
                if y[j - 1][k - 1].x == 1:
                    onPlayer[aGame[g]["date"]][playerName[g][j - 1]]["bat"] = k

        print("===== complete game", g, "- stage 2 =====")
        stage2Obj.append(model2.objVal)

    return onPlayer, stage2Obj


### Optimization

In [114]:
aTeam = "富邦悍將"
aMonth = "Apr"
aGame = scheduleByWeek[8]

In [115]:
playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
onPlayer = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
onPlayer, stage2Obj = stage2(aTeam, aGame, onPlayer)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 756 rows, 608 columns and 1939 nonzeros
Model fingerprint: 0x83214d84
Variable types: 0 continuous, 608 integer (608 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 2302.6753258
Presolve removed 745 rows and 593 columns
Presolve time: 0.00s
Presolved: 11 rows, 15 columns, 39 nonzeros
Found heuristic solution: objective 3509.6346404
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 3.523628e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3523.6275317 3523.62753  0.00%   

In [116]:
print(stage1Obj, stage1ObjWeight)
print(stage2Obj)

3523.627531711645 769.3509894566911
[3.136, 3.045, 3.103, 3.103, 3.103]


In [117]:
onPlayer

{datetime.date(2022, 5, 17): {'張進德': {'field': '2', 'bat': 6},
  '范國宸': {'field': '3', 'bat': 2},
  '葉竹軒': {'field': '4', 'bat': 1},
  '董子恩': {'field': '5', 'bat': 9},
  '李宗賢': {'field': '6', 'bat': 4},
  '陳真': {'field': '7', 'bat': 5},
  '申皓瑋': {'field': '8', 'bat': 3},
  '王正棠': {'field': '9', 'bat': 7},
  '蔣智賢': {'field': 'DH', 'bat': 8}},
 datetime.date(2022, 5, 18): {'林琨笙': {'field': '2', 'bat': 2},
  '蔣智賢': {'field': '3', 'bat': 3},
  '葉竹軒': {'field': '4', 'bat': 4},
  '董子恩': {'field': '5', 'bat': 5},
  '李宗賢': {'field': '6', 'bat': 7},
  '陳真': {'field': '7', 'bat': 9},
  '申皓瑋': {'field': '8', 'bat': 1},
  '范國宸': {'field': '9', 'bat': 8},
  '張進德': {'field': 'DH', 'bat': 6}},
 datetime.date(2022, 5, 20): {'張進德': {'field': '2', 'bat': 6},
  '蔣智賢': {'field': '3', 'bat': 2},
  '葉竹軒': {'field': '4', 'bat': 1},
  '董子恩': {'field': '5', 'bat': 9},
  '李宗賢': {'field': '6', 'bat': 4},
  '陳真': {'field': '7', 'bat': 5},
  '申皓瑋': {'field': '8', 'bat': 3},
  '范國宸': {'field': '9', 'bat': 7},
  '張冠